In [2]:
import os, re
import PIL.Image

from keras.preprocessing.image import ImageDataGenerator, img_to_array

Using TensorFlow backend.


tf.estimator package not installed.


In [4]:
def check_exists(path):
    return os.path.exists(path)

def check_dir(path, create = True):
    if os.path.isdir(path):
        return True
    else:
        if create:
            os.makedirs(path)
            return True
        else:
            return False

def check_file(file, create = True):
    if os.path.isfile(file):
        return True
    else:
        if create:
            directory, filename = os.path.split(file)
            check_dir(directory, True)
            open(file, 'w').close()
            return True
        else:
            return False

def get_files(loc_data):
    rex = re.compile(r'^.*\.({})$'.format('jpg|jpeg|bmp|png'), re.I)
    files = [os.path.join(loc_data,base) for base in os.listdir(loc_data) if rex.match(base)]
#     print(files)
    return files

def gen_preview(files,to,count=1,prefix = 'preview', size = (300,300),extension = 'jpg'):
    '''takes files and create store them to `to` folder'''
    
    if not os.path.isdir(to):
        os.makedirs(to)
    
    datagen = None
    if count is 1:
        datagen = ImageDataGenerator()
    else:
        datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')
    
#     bar = progressbar.ProgressBar(maxval=20, 
#                                   widgets=[progressbar.Bar('=', '[', ']'), ' ',
#                                            progressbar.Percentage()])
#     bar.start()
    for file in files:
        img = PIL.Image.open(file).resize(size,3)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir=to, save_prefix=prefix, save_format=extension):
            i+=1
            if i >= count:
                break
#     bar.finish()            
    print('Preview successfully created')
    
def divide_files(source_path, destination_path, percent = 30, shuffle = True):
    files = get_files(source_path)
    check_dir(destination_path,True)
    num_validation = (percent*len(files))//100
    print('Valdation File count :', num_validation)
    
    if shuffle:
        import random
        random.shuffle(files)
    
    for file in files[0:num_validation]:
        _, filename = os.path.split(file)
        os.rename(file, destination_path + '/' + filename)
        
def change_extention(files, _from = ['jpg','jpeg','png','bmp'], to = 'jpg'):
    for file in files:
        try:
            new_file, extention = file.rsplit('.',1)
            if extention in _from:
                os.rename(file, new_file + '.' + to)
        except:
            print('skiping :',file)

In [77]:
files = get_files('../upload/1')
print(len(files))

421


In [69]:
divide_files('../upload/1','../upload/6',20)

Valdation File count : 105


In [59]:
change_extention(files, to = 'png')

In [27]:
gen_preview(files,'test/set1/other',1)

Preview successfully created
